In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My\ Drive

/content/gdrive/My Drive


In [0]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np # MATRIX OPERATIONS
import pandas as pd # EFFICIENT DATA STRUCTURES
import cv2 as cv # IMAGE PROCESSING - OPENCV


# KERAS AND SKLEARN MODULES
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,CSVLogger

In [0]:
#READING TRAIN, TEST, GROUND_TRUVTH, SAMPLE_SUBMISSION
train = pd.read_csv("ikea-classification/Train/train.csv")
test = pd.read_csv("ikea-classification/test.csv")
ground_truth = pd.read_csv("ikea-classification/ground_truth.csv")
sample_submission = pd.read_csv("ikea-classification/Sample_Submission.csv")

In [0]:
#READING TRAIN IMAGES AND RESIZING THEM
temp = []
for img_name in train.filename:
  image_path = os.path.join("ikea-classification/Train/images/train", str(img_name))
  img = cv.imread(image_path)
  img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
  img = cv.resize(img,(70,70),interpolation=cv.INTER_AREA)
  temp.append(img)

train_x = np.stack(temp)

train_x = train_x.reshape(train_x.shape[0], 70, 70, 1).astype('float32')
train_x /= 255.0

In [0]:
#READING TEST IMAGES AND RESIZING THEM
temp = []
for img_name in test.filename:
  image_path = os.path.join("ikea-classification/Train/images/test", str(img_name))
  img = cv.imread(image_path)
  img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
  img = cv.resize(img,(70,70),interpolation=cv.INTER_AREA)
  temp.append(img)

test_x = np.stack(temp)

test_x = test_x.reshape(test_x.shape[0], 70, 70, 1).astype('float32')
test_x /= 255.0

In [0]:
train_y = keras.utils.np_utils.to_categorical(train.label.values)
split_size = int(train_x.shape[0]*0.7)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train_y[:split_size], train_y[split_size:]

In [0]:
# reshape data

train_x_temp = train_x.reshape(train_x.shape[0], 70, 70, 1)
val_x_temp = val_x.reshape(val_x.shape[0], 70, 70, 1)

In [0]:
generator = ImageDataGenerator(rotation_range = 180,zoom_range = 0.1,width_shift_range = 0.1,height_shift_range = 0.1,horizontal_flip = True,vertical_flip = True)
generator.fit(train_x_temp)

In [17]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(2, 2), input_shape=(70, 70, 1), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=64, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))

model.add(Conv2D(filters=128, kernel_size=(2, 2), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=128, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))


model.add(Conv2D(filters=256, kernel_size=(2, 2), activation='relu'))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(filters=256, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 69, 69, 64)        320       
_________________________________________________________________
batch_normalization_9 (Batch (None, 69, 69, 64)        256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 68, 68, 64)        16448     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 34, 34, 64)        0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 34, 34, 64)        256       
_________________________________________________________________
dropout_6 (Dropout)          (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 33, 33, 128)       32896     
__________

In [18]:
# SETTING UP CHECKPOINTS, CALLBACKS AND REDUCING LEARNING RATE
lrr = ReduceLROnPlateau(monitor='val_acc', 
                        patience=3, 
                        verbose=1, 
                        factor=0.4, 
                        min_lr=0.00001)

filepath="ikea-classification/model_weights/weights.best_{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoints = ModelCheckpoint(filepath, monitor='val_acc', 
                              verbose=1, save_best_only=True, mode='max')
filepath="ikea-classification/model_weights/weights.last_auto4.hdf5"
checkpoints_full = ModelCheckpoint(filepath, monitor='val_acc', 
                                 verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoints, lrr, checkpoints_full]

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
trained_model_conv = model.fit_generator(generator.flow(train_x_temp, train_y, batch_size=75), 
                           epochs=35, validation_data=(val_x_temp, val_y), 
                           steps_per_epoch=train_x_temp.shape[0], callbacks=callbacks_list)



# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# trained_model_conv = model.fit(train_x_temp, train_y, nb_epoch=epochs, batch_size=batch_size, validation_data=(val_x_temp, val_y))

Epoch 1/35
814/814 [==============================] - 67s 83ms/step - loss: 0.6201 - acc: 0.7706 - val_loss: 6.6881 - val_acc: 0.0000e+00

Epoch 00001: val_acc improved from -inf to 0.00000, saving model to ikea-classification/weights.best_01-0.00.hdf5

Epoch 00001: saving model to ikea-classification/weights.last_auto4.hdf5
Epoch 2/35
814/814 [==============================] - 66s 81ms/step - loss: 0.3162 - acc: 0.8792 - val_loss: 11.1300 - val_acc: 0.0000e+00

Epoch 00002: val_acc did not improve from 0.00000

Epoch 00002: saving model to ikea-classification/weights.last_auto4.hdf5
Epoch 3/35
814/814 [==============================] - 66s 82ms/step - loss: 0.2062 - acc: 0.9236 - val_loss: 5.4876 - val_acc: 0.3381

Epoch 00003: val_acc improved from 0.00000 to 0.33811, saving model to ikea-classification/weights.best_03-0.34.hdf5

Epoch 00003: saving model to ikea-classification/weights.last_auto4.hdf5
Epoch 4/35
814/814 [==============================] - 66s 81ms/step - loss: 0.1372 

In [0]:
pred = model.predict_classes(test_x)

In [0]:
y_actual = ground_truth.label.values

In [21]:
#Compute the confusion matrices
cmatrix = confusion_matrix(y_actual, pred)
print(cmatrix)

[[15 41  5  3]
 [ 0 62  0  0]
 [ 0  0 26  0]
 [ 0  0  0 48]]


In [0]:
sample_submission.filename = test.filename; sample_submission.label = pred
sample_submission.to_csv('ikea-classification/Submission.csv', index=False)